Imports

In [1]:
import pandas as pd
import random
import numpy as np
import warnings

Global functions

In [3]:
def get_accident_df(subset="standard_small", n_rows=10000, is_refined=False):
    warnings.warn("This function is outdated and should not be used.")
    if is_refined:
        df = get_accident_df(subset=subset, n_rows=n_rows, is_refined=False)
        return refine_df(df)

    if subset == "standard_small":
        return pd.read_csv("DataSubsetRaw.csv")
    if subset == "new":
        n = 3513617 #number of rows in the dataset
        s = n_rows #number of rows to sample
        skip = sorted(random.sample(range(1, n+1), n-s)) #rows to be skipped
        return pd.read_csv("US_Accidents_June20.csv", skiprows=skip)
    if subset == "all":
        return pd.read_csv("US_Accidents_June20.csv")

Internal functions

In [4]:
def refine_df(df):
    df = change_column_names(df)
    df = convert_to_metric(df)
    return df

In [5]:
def change_column_names(df):
    col_names = df.columns
    col_names = [name.replace("_", "") for name in col_names]
    col_names = [name[:loc] if (loc := name.find("(")) != -1 else name for name in col_names]
    df.columns = col_names
    return df

In [ ]:
def convert_to_metric(df):
    #miles to km
    df["Distance"] = df["Distance"] * 1.60934

    #F to C
    df["Temperature"] = (df["Temperature"] - 32) * 5 / 9

    #F to C
    df["WindChill"] = (df["WindChill"] - 32) * 5 / 9

    #inches of mercury to bar
    df["Pressure"] = df["Pressure"] / 29.53

    #miles to km
    df["Visibility"] = df["Visibility"] * 1.60934

    #mph to kph
    df["WindSpeed"] = df["WindSpeed"] * 1.60934

    #inches to cm
    df["Precipitation"] = df["Precipitation"] * 2.54
    return df

Testing